## CHAPTER 9
# Data Aggregation and Group Operations
---
## Group-wise

In [2]:
%pylab inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.DataFrame({'data1': np.random.randn(5), 'data2': np.random.randn(5),
                   'key1': ['a','a','b','b','a'], 'key2': ['one','two','one','two','one']})
df

,data1,data2,key1,key2
0,0.054933,0.154873,a,one
1,-1.401412,-0.873075,a,two
2,0.966912,0.326601,b,one
3,-0.495702,-0.706336,b,two
4,-0.418913,0.878013,a,one


In [4]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,-0.588464,0.053270
b,0.235605,-0.189868


In [5]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0.054933,0.154873,a,one,-0.588464,0.053270
1,-1.401412,-0.873075,a,two,-0.588464,0.053270
4,-0.418913,0.878013,a,one,-0.588464,0.053270
2,0.966912,0.326601,b,one,0.235605,-0.189868
3,-0.495702,-0.706336,b,two,0.235605,-0.189868


In [6]:
people = pd.DataFrame(np.random.randn(5, 5),
    columns=['a', 'b', 'c', 'd', 'e'],
    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2][['b','c']]=None
display(people)

,a,b,c,d,e
Joe,-0.348218,0.239592,0.615169,-1.009688,-0.301234
Steve,0.908614,-1.039960,1.584468,-0.330935,-0.190611
Wes,-0.784900,NaN,NaN,-0.817304,-0.337738
Jim,-1.341105,1.151901,-0.349238,1.425554,-0.976384
Travis,1.237226,-0.720527,-0.326408,0.617443,-1.276180


In [7]:
key = ['one','two','one','two','one']
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,0.034702,-0.240468,0.144381,-0.403183,-0.638384
Steve,-0.216246,0.055970,0.617615,0.547309,-0.583497
Wes,0.034702,-0.240468,0.144381,-0.403183,-0.638384
Jim,-0.216246,0.055970,0.617615,0.547309,-0.583497
Travis,0.034702,-0.240468,0.144381,-0.403183,-0.638384


In [8]:
people.groupby(key).transform(lambda arr: arr-arr.mean())

,a,b,c,d,e
Joe,-0.382921,0.480059,0.470789,-0.606505,0.337150
Steve,1.124860,-1.095931,0.966853,-0.878245,0.392887
Wes,-0.819603,NaN,NaN,-0.414121,0.300646
Jim,-1.124860,1.095931,-0.966853,0.878245,-0.392887
Travis,1.202523,-0.480059,-0.470789,1.020626,-0.637796


In [9]:
help(people.groupby(key).transform)

Help on method transform in module pandas.core.groupby.groupby:

transform(func, *args, **kwargs) method of pandas.core.groupby.groupby.DataFrameGroupBy instance
    Call function producing a like-indexed DataFrame on each group and
    return a DataFrame having the same indexes as the original object
    filled with the transformed values
    
    Parameters
    ----------
    f : function
        Function to apply to each group
    
    Notes
    -----
    Each group is endowed the attribute 'name' in case you need to know
    which group you are working on.
    
    The current implementation imposes three requirements on f:
    
    * f must return a value that either has the same shape as the input
      subframe or can be broadcast to the shape of the input subframe.
      For example, f returns a scalar it will be broadcast to have the
      same shape as the input subframe.
    * if this is a DataFrame, f must support application column-by-column
      in the subframe. If f als

### [주의] transform() requirements
- the passed function must either produce a scalar value or a transformed array of the same size

## apply()
- make pieces
- then concaternate

In [10]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips.tip / tips.total_bill
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [11]:
tips.sort_values('tip_pct',ascending=False).head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535


In [12]:
tips.groupby(['sex','smoker']).apply(lambda df: df.sort_values('tip_pct',ascending=False)[:5])

total_bill   tip     sex smoker   day    time  size  \
sex    smoker                                                            
Female No     51        10.29  2.60  Female     No   Sun  Dinner     2   
              139       13.16  2.75  Female     No  Thur   Lunch     2   
              18        16.97  3.50  Female     No   Sun  Dinner     3   
              14        14.83  3.02  Female     No   Sun  Dinner     2   
              115       17.31  3.50  Female     No   Sun  Dinner     2   
       Yes    178        9.60  4.00  Female    Yes   Sun  Dinner     2   
              67         3.07  1.00  Female    Yes   Sat  Dinner     1   
              109       14.31  4.00  Female    Yes   Sat  Dinner     2   
              93        16.32  4.30  Female    Yes   Fri  Dinner     2   
              221       13.42  3.48  Female    Yes   Fri   Lunch     2   
Male   No     232       11.61  3.39    Male     No   Sat  Dinner     2   
              149        7.51  2.00    Male     No  Thur   Lunch     2   
              185       20.69  5.00    Male     No   Sun  Dinner     5   
              88        24.71  5.85    Male     No  Thur   Lunch     2   
              6          8.77  2.00    Male     No   Sun  Dinner     2   
       Yes    172        7.25  5.15    Male    Yes   Sun  Dinner     2   
              183       23.17  6.50    Male    Yes   Sun  Dinner     4   
              181       23.33  5.65    Male    Yes   Sun  Dinner     2   
              194       16.58  4.00    Male    Yes  Thur   Lunch     2   
              174       16.82  4.00    Male    Yes   Sun  Dinner     2   

                    tip_pct  
sex    smoker                
Female No     51   0.252672  
              139  0.208967  
              18   0.206246  
              14   0.203641  
              115  0.202195  
       Yes    178  0.416667  
              67   0.325733  
              109  0.279525  
              93   0.263480  
              221  0.259314  
Male   No     232  0.291990  
              149  0.266312  
              185  0.241663  
              88   0.236746  
              6    0.228050  
       Yes    172  0.710345  
              183  0.280535  
              181  0.242177  
              194  0.241255  
              174  0.237812

In [13]:
tips.groupby(['sex','smoker'],group_keys=False).apply(lambda df: df.sort_values('tip_pct',ascending=False)[:5])

,total_bill,tip,sex,smoker,day,time,size,tip_pct
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
139,13.16,2.75,Female,No,Thur,Lunch,2,0.208967
18,16.97,3.50,Female,No,Sun,Dinner,3,0.206246
14,14.83,3.02,Female,No,Sun,Dinner,2,0.203641
115,17.31,3.50,Female,No,Sun,Dinner,2,0.202195
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
93,16.32,4.30,Female,Yes,Fri,Dinner,2,0.263480
221,13.42,3.48,Female,Yes,Fri,Lunch,2,0.259314


In [14]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(column,ascending=False)[:n]

tips.groupby(['smoker','day']).apply(top,n=2,column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
            91        22.49   3.50    Male     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
            59        48.27   6.73    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
            112       38.07   4.00    Male     No   Sun  Dinner     3   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
            85        34.83   5.17  Female     No  Thur   Lunch     4   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
            90        28.97   3.00    Male    Yes   Fri  Dinner     2   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
            102       44.30   2.50  Female    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
            184       40.55   3.00    Male    Yes   Sun  Dinner     2   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
            83        32.68   5.00    Male    Yes  Thur   Lunch     2   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
            91   0.155625  
       Sat  212  0.186220  
            59   0.139424  
       Sun  156  0.103799  
            112  0.105070  
       Thur 142  0.121389  
            85   0.148435  
Yes    Fri  95   0.117750  
            90   0.103555  
       Sat  170  0.196812  
            102  0.056433  
       Sun  182  0.077178  
            184  0.073983  
       Thur 197  0.115982  
            83   0.152999

In [15]:
tips.groupby(['smoker']).describe()

size                                                 tip            \
        count      mean       std  min  25%  50%  75%  max  count      mean   
smoker                                                                        
No      151.0  2.668874  1.017984  1.0  2.0  2.0  3.0  6.0  151.0  2.991854   
Yes      93.0  2.408602  0.810751  1.0  2.0  2.0  3.0  5.0   93.0  3.008710   

        ...     tip_pct           total_bill                             \
        ...         75%       max      count       mean       std   min   
smoker  ...                                                               
No      ...    0.185014  0.291990      151.0  19.188278  8.255582  7.25   
Yes     ...    0.195059  0.710345       93.0  20.756344  9.832154  3.07   

                                      
           25%    50%     75%    max  
smoker                                
No      13.325  17.59  22.755  48.33  
Yes     13.420  17.92  26.860  50.81  

[2 rows x 32 columns]

In [16]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,-0.908272,-1.373422
1,-0.061746,-2.938430
2,-0.676057,-0.936429
3,-1.782162,0.127515
4,-0.578072,-0.748498


In [17]:
frame.data1.max(), frame.data1.min()

(2.8616920532950254, -3.016635536650859)

In [18]:
pd.cut(frame.data1,4).head()

0    (-1.547, -0.0775]
1     (-0.0775, 1.392]
2    (-1.547, -0.0775]
3     (-3.023, -1.547]
4    (-1.547, -0.0775]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.023, -1.547] < (-1.547, -0.0775] < (-0.0775, 1.392] < (1.392, 2.862]]

In [19]:
# frame.data2.groupby(pd.cut(frame.data1,4)).agg(['mean','count'])
frame.groupby(pd.cut(frame.data1,4)).agg(['mean','count'])

data1           data2      
                       mean count      mean count
data1                                            
(-3.023, -1.547]  -1.946534    62  0.012720    62
(-1.547, -0.0775] -0.659510   385  0.002056   385
(-0.0775, 1.392]   0.538715   466  0.063615   466
(1.392, 2.862]     1.786106    87  0.062728    87

In [20]:
frame.groupby(pd.qcut(frame.data1,4)).agg(['mean','count'])

data1           data2      
                      mean count      mean count
data1                                           
(-3.018, -0.602] -1.240919   250 -0.065393   250
(-0.602, 0.0352] -0.259795   250  0.078823   250
(0.0352, 0.708]   0.345736   250  0.073416   250
(0.708, 2.862]    1.282320   250  0.059882   250

In [21]:
l = []
for suit in list('HSCD'):
    for num in ['A']+list(range(2,11))+list('JKQ'):
        l.append([str(num)+suit,1 if num=='A' else (10 if str(num) in 'JKQ' else num)])
# deck=pd.DataFrame(l).set_index(0)
deck=pd.Series([i[1] for i in l], index=[i[0] for i in l])
deck.name='card_value'
deck.index.name='card'
deck.head(13)

card
AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
Name: card_value, dtype: int64

In [22]:
deck[np.random.permutation(len(deck))[:10]]

card
KC    10
JC    10
6D     6
AD     1
JS    10
4H     4
QH    10
3D     3
5S     5
8H     8
Name: card_value, dtype: int64

In [23]:
deck.groupby(lambda s: s[-1]) # groupby 인자로 함수를 넘기면, index 에 대해 적용된다
deck.groupby(lambda s: s[-1]).apply(lambda ser: ser[np.random.permutation(len(ser))[:2]])
# deck.groupby(lambda s: s[-1],group_keys=False).apply(lambda ser: ser[np.random.permutation(len(ser))[:2]])
# deck.groupby(lambda s: s[-1]).apply(lambda ser: ser[np.random.permutation(len(ser))[:2]]).reset_index(level=0, drop=True)

   card
C  KC      10
   AC       1
D  8D       8
   2D       2
H  AH       1
   2H       2
S  6S       6
   9S       9
Name: card_value, dtype: int64